(databases)=
# Databases

## Introduction

A huge amount of data lives in databases, so it's essential that you know how to access them. Sometimes you can ask someone to download a snapshot into a .csv for you, but this gets painful quickly: every time you need to make a change you'll have to communicate with another human. You want to be able to reach into the database directly to get the data you need, when you need it.

In this chapter, you'll first learn the basics of Python packages that can interact with SQL databses: how to use them to connect to a database and then retrieve data with a SQL[^import-databases-1] query. **SQL**, short for **s**tructured **q**uery **l**anguage, is the lingua franca of databases, and is an important language for all data scientists to learn. You won't become a SQL master by the end of the chapter, but you will be able to identify the most important components and understand what they do.

[^import-databases-1]: SQL is either pronounced "s"-"q"-"l" or "sequel".

### Prerequisites

You will need the **pandas** and **SQLModel** packages for this chapter. You probably already have **pandas** installed; to install **SQLModel**, run `pip install sqlmodel` on your computer's command line.

## Database Basics

At the simplest level, you can think about a database as a collection of data frames, called **tables** in database terminology.
Like a **pandas** dataframe, a database table is a collection of named columns, where every value in the column is the same type.
There are three high level differences between data frames and database tables:

-   Database tables are stored on disk (ie on file) and can be arbitrarily large.
    Data frames are stored in memory, and are fundamentally limited (although that limit is still big enough for many problems). You can think about the difference between on disk and in memory as being like the difference between long-term and short-term memory (and you have much more limited capacity in the latter).

-   Database tables almost always have indexes.
    Much like the index of a book, a database index makes it possible to quickly find rows of interest without having to look at every single row.

-   Most classical databases are optimised for rapidly collecting data, not analysing existing data.
    These databases are called **row-oriented** because the data is stored row-by-row, rather than column-by-column.
    More recently, there's been much development of **column-oriented** databases that make analysing the existing data much faster.

Databases are run by database management systems (**DBMS**'s for short), which come in three basic forms:

-   *Client-server* DBMS's run on a powerful central server, which you connect from your computer (the client). They are great for sharing data with multiple people in an organisation. Popular client-server DBMS's include PostgreSQL, MariaDB, SQL Server, and Oracle.
-   *Cloud* DBMS's, like Snowflake, Amazon's RedShift, and Google's BigQuery, are similar to client server DBMS's, but they run in the cloud. This means that they can easily handle extremely large datasets and can automatically provide more compute resources as needed.
-   *In-process* DBMS's, like SQLite or duckdb, run entirely on your computer. They're great for working with large datasets where you're the primary user.

## Connecting to Databases

To connect to a database from Python, there are a number of options, but it does depend on what *kind* of database you are connecting to. The steps (eg connect to the database, upload data, run SQL series) are similar though, so we won't lose much by just choosing one example—a SQLite database. SQLite is a small, fast, self-contained, SQL database engine and the most used database engine in the world. A lot of the data on your computer and mobile phone will in fact be in SQLite databases. (Simon Willison has some [great tools](https://dogsheep.github.io/) to help export some of the data that you've created!)

We will always need two steps though, regardless of what kind of SQL database we are connecting to:

-   You'll always use a database interface that provides a connection to the database, for example Python's built-in **sqlite** package

-   You'll also use a package that pushes and/or pulls data to/from the database, for example **pandas**

The precise details of the connection varies a lot from DBMS to DBMS so unfortunately we can't cover all the details here. The initial setup will often take a little fiddling (and maybe some research) to get right, but you'll generally only need to do it once. We'll do the best we can to cover some basics here.

Setting up a client-server or cloud DBMS would be a pain for this book, so we'll instead use an in-computer DBMS.

## Working with Databases Directly

Let's connect to a small SQLite database called the [Chinook database](https://database.guide/2-sample-databases-sqlite/), which contains information about the artists, songs, and albums from a music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. The figure below shows the data schema:

![Data schema of Chinook database](https://github.com/arjunchndr/Analyzing-Chinook-Database-using-SQL-and-Python/raw/master/chinook-schema.svg)

You can download the file, Chinook.sqlite, from the github repository of this book [here](https://github.com/aeturrell/python4DS/tree/main/data); on your computer, you'll need to save it in a subdirectory (relative to where your code is being executed) called 'data' to work through the exercises below.

For starters, let's use Python's built in **sqlite3** engine to connect to the database and execute a very simple SQL query to select the first ten entries from the 'Artists' table:

In [ ]:
import sqlite3

con = sqlite3.connect("data/Chinook.sqlite")

cursor = con.execute("SELECT * FROM Artist LIMIT 10;")
rows = cursor.fetchall()
rows

Note that the output here is in the form a Python object called a tuple. If we wanted to put this into a **pandas** dataframe, we can just pass it straight in:

In [ ]:
import pandas as pd

pd.DataFrame(rows)

Another useful hint is that if you're not sure what the column names are, you can obtain them from:

In [ ]:
[i[0] for i in cursor.description]

### Creating a Database

Often, you'll want to create a SQL database to later (efficiently) access cuts of data. Let's create a test database directly using the **sqlite** package. This process involves a `CREATE TABLE` statement, then the name of the table followed by the names of the columns and their data types.

In [ ]:
create_query = """CREATE TABLE test (country VARCHAR(20), gdp REAL, health INTEGER);"""
con_new = sqlite3.connect("data/test_database.sqlite")
con_new.execute(create_query)
con_new.commit()

If this runs, then you've created a test database! (You can check the data directory on your own computer to see if it worked, but you'll get an error if the data directory doesn't already exist.)

Let's now fill the database with some values:

In [ ]:
test_data = [("US", 1, 3), ("UK", 0.6, 2), ("France", 0.8, 1)]

con_new.executemany("INSERT INTO test VALUES(?, ?, ?)", test_data)
con_new.commit()

Finally, let's check if this has worked:

In [ ]:
con_new.execute("SELECT * FROM test").fetchall()

Whahey, this worked!

In [ ]:
# remove-cell

import os

os.remove("data/test_database.sqlite")

## Basic SQL Queries

A full description of SQL queries is outside the scope of this book, but we'll try and arm you with the basics.  The top-level components of SQL are called **statements**. Common statements include `CREATE` for defining new tables, `INSERT` for adding data, and `SELECT` for retrieving data. We will on focus on `SELECT` statements, also called **queries**, because they are almost exclusively what you'll use as a data scientist.

A query is made up of **clauses**. There are six important clauses: `SELECT`, `FROM`, `WHERE`, `ORDER BY`, `GROUP BY`, and `LIMIT`. Every query must have the `SELECT`[^import-databases-4] and `FROM`[^import-databases-5] clauses and the simplest query is `SELECT * FROM table`, which selects all columns from a specified table called "table". `WHERE` and `ORDER BY` control which rows are included and how they are ordered. `GROUP BY` converts the query to a summary, causing aggregation to happen. `LIMIT` limits how many rows are returned.

Important: in SQL, the order matters. You must always write the clauses in the order `SELECT`, `FROM`, `WHERE`, `GROUP BY`, `ORDER BY`. Confusingly, this order doesn't match how the clauses actually evaluated which is first `FROM`, then `WHERE`, `GROUP BY`, `SELECT`, and `ORDER BY`.


[^import-databases-4]: Confusingly, depending on the context, `SELECT` is either a statement or a clause.
    To avoid this confusion, we'll generally use query instead of `SELECT` statement.

[^import-databases-5]: Technically, only the `SELECT` is required, since you can write queries like `SELECT 1+1` to perform basic calculations. But if you want to work with data (as you always do!) you'll also need a `FROM` clause.

Let's look at a few of these in action. To get a few entries, it's

```sql
SELECT * FROM Artist LIMIT 10;
```

To be specific about columns, it's

```sql
SELECT name FROM Artist LIMIT 10;
```

To reverse order, use "ORDER BY" and then "DESC". For example, this query gives us the ten longest tracks

```sql
SELECT name, milliseconds FROM track ORDER BY milliseconds DESC LIMIT 10;
```

Another headline feature is being able to filter. For example, we could ask for the first ten tracks that are at least 3 minutes long. Let's run this one for real:

In [ ]:
sql_query = "SELECT name, milliseconds FROM track WHERE milliseconds > 1e3*3*60 ORDER BY milliseconds ASC LIMIT 10;"
cursor = con.execute(sql_query)
rows = cursor.fetchall()
rows

Let's try a groupby. This one will groupby album to find the average track length in minutes for (the first five) albums:

In [ ]:
sql_groupby = "SELECT albumid, AVG(milliseconds)/1e3/60 FROM track GROUP BY albumid ORDER BY AVG(milliseconds) ASC LIMIT 5;"
cursor = con.execute(sql_groupby)
rows = cursor.fetchall()
rows

### Joins

If you're familiar with joins in **pandas**, SQL joins are very similar. Let's see if we can join the 'album' and 'track' tables to find the *name* of the albums in the above query.

Note that as soon as we have the *same* column names in more than one table, we need to specify the table we are referring to when we use that column name. There are different options for joins (eg `INNER`, `LEFT`) that you can find out more about [here](https://en.wikipedia.org/wiki/Join_(SQL)).


In [ ]:
sql_join = "SELECT track.albumid, AVG(milliseconds)/1e3/60, album.title FROM track INNER JOIN album ON (track.albumid = album.albumid) GROUP BY album.albumid ORDER BY AVG(milliseconds) ASC LIMIT 5;"
cursor = con.execute(sql_join)
rows = cursor.fetchall()
rows

### Functions

We already snuck in one function: `AVG`. Others that you can apply to columns include `SUM`, `MIN`, `MAX`, `FIRST`, `LAST` and `COUNT` (the data scientist's friend). You can find out more about SQL functions [here](https://www.tutorialspoint.com/sql/sql-useful-functions.htm).

## SQL with **pandas**

**pandas** is well-equipped for working with SQL. We can simply push the query we just created straight through using its `read_sql` function—but bear in mind we need to pass in the connection we created to the database too:

In [ ]:
pd.read_sql(sql_join, con)

One nice feature of this is that the column names in SQL get passed straight to the column names in our dataframe.

## SQL with **SQLModel**

Now, it's not exactly satisfactory to have to write out your SQL queries in text. When you're writing Python in Visual Studio Code (at least with the Python extensions installed), you get a lot of high quality syntax and auto-completion support. Extensions to the Python language also allow you to take a great deal of care over the types of variables that you are dealing with. Wouldn't it be nice to have all of that with SQL too (even when accessing it via Python)?

[**SQLModel**](https://sqlmodel.tiangolo.com/) brings some of those benefits, making working with SQL databases from Python a lot easier and more productive.

First, make sure you have the package installed by running `pip install sqlmodel` on the command line.

### Creating a SQL Database with **SQLModel**

Imagine you have a SQL table called "hero" and you want it to have this data:

| id | name | secret_name | age |
|---|---|---|---|
| 1 | Deadpond | Dive Wilson | null |
| 2 | Spider-Boy | Pedro Parqueador | null |
| 3 | Rusty-Man | Tommy Sharp | 48 |
| 4 | Ms Amazing | Barjabeen Bhabra | 17 |

Now, to create the SQL table, we write a structure called a *class*. Classes look a bit like functions but, in the body of this class, we list the name of the field followed by the Python code for the *type* of data represented:

In [ ]:
# remove-cell

import warnings

warnings.filterwarnings(
    "ignore", ".*Class SelectOfScalar will not make use of SQL compilation caching.*"
)

In [ ]:
from typing import Optional

from sqlmodel import Field, Session, SQLModel, create_engine


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

We then create the rows of data (this could be done programmatically but we'll write them out here). Note that there is autocompletion of the fields in the below!

In [ ]:
hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
hero_2 = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
hero_3 = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=48)
hero_4 = Hero(name="Ms Amazing", secret_name="Barjabeen Bhabra", age=17)

Now we need to create the actual database and connect to it. The syntax is the name of the type of database, here sqlite, and then `:///` followed by the relative path to where you want to store the database.

In [ ]:
engine = create_engine("sqlite:///data/hero.db")

SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    session.add(hero_1)
    session.add(hero_2)
    session.add(hero_3)
    session.add(hero_4)
    session.commit()

We can now get a read-out from the database we created using the direct approach (ie using the **sqlite3** package):

In [ ]:
con = sqlite3.connect("data/hero.db")

con.execute("SELECT * FROM hero;").fetchall()

### Reading from SQL using **SQLModel**

**SQLModel** translates all of the usual SQL statements into functions. "SELECT" is no different. So let's see an example of reading the SQL database directly from **SQLModel**:

In [ ]:
from sqlmodel import select

with Session(engine) as session:
    statement = select(Hero)  # Equivalent to SELECT * from hero
    results = session.exec(statement)  # Execute the command
    for hero in results:
        print(hero)

We can also get all of it back via `results.all()`. The resulting object is a list of "hero" objects whose properties can be accessed via `heroes[0].name` and so on.

In [ ]:
with Session(engine) as session:
    statement = select(Hero)
    results = session.exec(statement)
    heroes = results.all()

print(*heroes, sep="\n")

You can find more info in the [documentation](https://sqlmodel.tiangolo.com/) for **SQLModel**, but most of the SQL query commands you'd expect are available. Let's run an example from earlier:

In [ ]:
with Session(engine) as session:
    statement = select(Hero).where(Hero.age < 100).limit(2).order_by(Hero.age)
    results = session.exec(statement)
    heroes = results.all()

print(*heroes, sep="\n")

This has barely scraped the surface of what **SQLModel** can do: joins, updating data, deleting rows, and more are all possible too.

In [ ]:
# remove-cell

os.remove("data/hero.db")

## Where to Learn More About SQL

If you've finished this chapter and would like to learn more about SQL, here are some further resources:

-   [*SQL for Data Scientists*](https://sqlfordatascientists.com) by Renée M. P. Teate is an introduction to SQL designed specifically for the needs of data scientists, and includes examples of the sort of highly interconnected data you're likely to encounter in real organisations.
-   [*Practical SQL*](https://www.practicalsql.com) by Anthony DeBarros is written from the perspective of a data journalist (a data scientist specialised in telling compelling stories) and goes into more detail about getting your data into a database and running your own DBMS.
-   Check out one of the many [*sql cheatsheets*](https://learnsql.com/blog/sql-basics-cheat-sheet/sql-basics-cheat-sheet-a3.pdf) that have all of the basics written in one place
-   Try out sql in your browser at [SQLime](https://sqlime.org/)
-   Learn SQL by solving a [murder mystery](https://mystery.knightlab.com/)